### ## DF CREATION

In [0]:
df_employees = spark.read.format("csv").option("inferSchema", True).option("Header", True).load("/Volumes/workspace/default/join/employees.csv")

df_department = spark.read.format("csv").option("inferschema", True).option("header", True).load("/Volumes/workspace/default/join/departments.csv")



#df_parquet = spark.read.format("parquet").load("/mnt/data/sample.parquet")

#df = spark.read.format("json").option("multiline", "true").load("/mnt/data/sample.json")



In [0]:
display(df_department)

### **JOINS/ remove duplicate column from DF

In [0]:
df_leftjoin = df_employees.join(df_department,df_employees.dept_id == df_department.dept_id, "left").drop(df_department.dept_id)
display(df_leftjoin)



In [0]:
df_rightjoin = df_employees.join(df_department, df_employees.dept_id == df_department.dept_id, "right")

display(df_rightjoin)

Broadcast join for data skewing (performance)

In [0]:
from pyspark.sql.functions import broadcast

df_broadcast = df_employees.join(broadcast(df_department), df_employees.dept_id == df_department.dept_id, "left").drop(df_employees.dept_id)

df_broadcast.write.format("delta").mode("overwrite").saveAsTable("workspace.default.test")


df_employees.write.format("csv").mode("overwrite").save("/Volumes/workspace/default/join/transform")



In [0]:
df_broadcast.write.format("csv").mode("overwrite").partitionBy("dept_name").save("/Volumes/workspace/default/join/test")

#display(df_broadcast)

In [0]:
df_uber = spark.read.format("csv").option("header", True).option("inferschema", True).load("/Volumes/workspace/default/uber")

### ** REGEXP_REPLACE VALUE REPLACE**

In [0]:
from pyspark.sql.functions import *

dfclean_uber = df_uber.withColumn("Booking ID", regexp_replace(col("Booking ID"), '"', '')) ## remove """""" from  "booking ID" column

df_clean = dfclean_uber.withColumn("Customer Rating", regexp_replace(col("Customer Rating"), "null", "0.0")) 
## change "customer Rating" "String" null to 0.0 to convert the string datatype to double

df_cast = df_clean.withColumn("Customer Rating",(col("Customer Rating").cast("double")))
 ## changed "customer Rating" column datatype from string  to double

df_ubercolumn = df_cast.withColumn("Bookingstar", when(col("Customer Rating") >= 4.5, "Good").otherwise("bad")) 
## creating new column "Booking Star" with data Good or bad  based on "Customer Rating" column



In [0]:
display(df_uber) #before regexp_replace
display(dfclean_uber) #after regexp_replace
